# Homework 4

Chris Dailey

christopher.dailey@gmail.com



In [ ]:
#imports


## HW 4.0
What is MrJob? How is it different to Hadoop MapReduce? 
What are the mappint_init, mapper_final(), combiner_final(), reducer_final() methods? When are they called?

## HW 4.1
What is serialization in the context of MrJob or Hadoop? 
When it used in these frameworks? 
What is the default serialization mode for input and outputs for MrJob? 


## HW 4.2

Preprocess the Microsoft log data into a format that can be analyzed in parallel

In [ ]:
inputFile = open("anonymous-msweb.data", 'r')

outputFile = open("cleanedlog.txt", 'w')

current_user = ""

for line in inputFile.readlines():
    